In [1]:
import os
import tweepy as tw
import pandas as pd
from geotext import GeoText
import nltk
import spacy
import locationtagger
import matplotlib.pyplot as plt
from branca.element import Figure
import folium


In [2]:
consumer_key = "L45cVwt1PcimfFnycMhQEHRsg"
consumer_secret = "a2f3dr6xHHukbCzlvzK5j0mDYg3di3aN8EfixXOhzV9dbhMk1F"
access_token = "1533339797187883008-A5uEjCSsKsH4OBeTaO8g0vHaqoI8h6"
access_token_secret = "UKvwBMeTRN6p4f2ogcrQAQT52Fi6eWUhe2pCnEkaJ8UGq"

In [3]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [4]:
# Define the search term and the date_since date as variables
search_words = "#wildfire OR #wildfires OR #forestfire OR #bushfire"
date_since = "2018-01-01"

In [5]:
new_search = search_words + " -filter:retweets"
new_search

'#wildfire OR #wildfires OR #forestfire OR #bushfire -filter:retweets'

In [6]:
tweets = tw.Cursor(api.search_tweets,
                       q=new_search,
                       lang="en").items(5000)


In [7]:
all_tweets = [[tweet.text] for tweet in tweets]


Rate limit reached. Sleeping for: 325


In [8]:
tweet_text = pd.DataFrame(data=all_tweets, 
                    columns=["tweet"])
                    
tweet_text

,tweet
0,@GabrielMariya @CopernicusEMS @EU_ScienceHub @...
1,Mediterranean #Wildfires in the Age of Climate...
2,@ann_annconquest @Norfolkfire I would encourag...
3,You want to understand how #drones are used in...
4,Specially marked fire protection canisters and...
5,The research team of Shanghai Jiao Tong Univer...
6,Dai Lanlan teaches Chinese at Weiming Primary ...
7,against #wildfires in SW #China's #Chongqing. ...
8,More than 1 month without using car 😊\nBetween...
9,Aerial view of residents in SW China's #Chongq...


In [9]:
df = tweet_text[tweet_text["tweet"].str.contains("RT") == False]
df

,tweet
0,@GabrielMariya @CopernicusEMS @EU_ScienceHub @...
1,Mediterranean #Wildfires in the Age of Climate...
2,@ann_annconquest @Norfolkfire I would encourag...
3,You want to understand how #drones are used in...
4,Specially marked fire protection canisters and...
5,The research team of Shanghai Jiao Tong Univer...
6,Dai Lanlan teaches Chinese at Weiming Primary ...
7,against #wildfires in SW #China's #Chongqing. ...
8,More than 1 month without using car 😊\nBetween...
9,Aerial view of residents in SW China's #Chongq...


In [10]:
df['tweet'] = df['tweet'].str.title()
df

,tweet
0,@Gabrielmariya @Copernicusems @Eu_Sciencehub @...
1,Mediterranean #Wildfires In The Age Of Climate...
2,@Ann_Annconquest @Norfolkfire I Would Encourag...
3,You Want To Understand How #Drones Are Used In...
4,Specially Marked Fire Protection Canisters And...
5,The Research Team Of Shanghai Jiao Tong Univer...
6,Dai Lanlan Teaches Chinese At Weiming Primary ...
7,Against #Wildfires In Sw #China'S #Chongqing. ...
8,More Than 1 Month Without Using Car 😊\nBetween...
9,Aerial View Of Residents In Sw China'S #Chongq...


In [11]:

df_countries = pd.DataFrame()


df_regions = pd.DataFrame()


df_cities = pd.DataFrame()


for index, row in df.iterrows():
    countries = row['tweet']
    place_entity = locationtagger.find_locations(text = countries)
    if place_entity.countries:
     df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
    if place_entity.regions:
     df_regions = df_regions.append({'region': place_entity.regions}, ignore_index=True)
    if place_entity.cities:
        for city_name in place_entity.cities:
            df_cities = df_cities.append({'city': city_name}, ignore_index=True)




/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2338329552.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2338329552.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_countries = df_countries.append({'country': place_entity.countries}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2338329552.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities = df_cities.append({'city': city_name}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2338329552.py:19: FutureWarning: The frame.append method

In [12]:
df_countries

,country
0,[China]
1,[China]


In [13]:
df_regions


""


In [14]:
df_cities

df_cities.drop_duplicates(subset=['city'], keep=False)

,city
0,Eu
5,Russia


In [15]:
from geopy.geocoders import Nominatim

loc = Nominatim(user_agent="cal")
print(loc)


In [16]:
df_cities_map = pd.DataFrame()

for ind in df_cities.index:
    getloc = loc.geocode(df_cities['city'][ind])
    df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)

df_cities_map=df_cities_map.dropna()
df_cities_map

/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_cities['city'][ind],'latitude': getloc.latitude, 'longitude': getloc.longitude}, ignore_index=True)
/var/folders/kp/bgyrqb3s3n19yxcx2cyqppwh0000gn/T/ipykernel_1108/2917749237.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_cities_map = df_cities_map.append({'city': df_citie

,city,latitude,longitude
0,Eu,50.049170,1.417574
1,Chongqing,29.564740,106.547877
2,China,35.000074,104.999927
3,Chongqing,29.564740,106.547877
4,China,35.000074,104.999927
5,Russia,64.686314,97.745306


In [17]:
# Creating Basemap
fig3=Figure(width=550,height=350)
m3=folium.Map(location=[-1.286389, 36.817223],tiles='cartodbpositron',zoom_start=1)
fig3.add_child(m3)

loc = 'Forest fire related 2000 tweets all around the world'
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   

m3.get_root().html.add_child(folium.Element(title_html))
#Adding markers to the map
for index, row in df_cities_map.iterrows():
    folium.Marker(location=[row['latitude'],row['longitude']],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(m3)


m3



In [18]:
from geopy.exc import GeocoderTimedOut

def put_markers(map, df_cities):
    geo_locator = Nominatim(user_agent="ForestFire")
    for index, row in df_cities_map.iterrows():
        if row['city']:
            try:
                location = geo_locator.geocode(row['city'])
            except GeocoderTimedOut:
                continue
            if location:
                folium.Marker([location.latitude, location.longitude],tooltip=row['city'],
        icon=folium.Icon(color="red", icon_color="white", icon="fire")).add_to(map)



In [19]:

map = folium.Map(location=[0, 0], zoom_start=2)
loc = 'Forest fire related 1000 tweets all around the world'
map.get_root().html.add_child(folium.Element(title_html))
title_html = '''
             <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(loc)   
location_data = df_cities
put_markers(map, location_data)
map.save("index.html")